<h2>개인 구글 드라이브와 colab 연동</h2>

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


<h2>CNN 모델</h2>

![대체 텍스트](http://nlp.kangwon.ac.kr/~nlpdemo/cnn.png)

In [0]:
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

class CNN(object):
  
  def __init__(self, flags):
    self.learning_rate = flags["learning_rate"]  # 학습률
    self.num_classes = flags["num_classes"]      # 분류할 클래스의 개수
    self.mode = flags["mode"]                    # 학습 or 평가 상태

    self._input_init()
    self._cnn_init()
    self._predict_init()
    self._train_init()

  # 입력 데이터, 정답 데이터, keep_prob 값을 담을 tensor 선언
  def _input_init(self):
    # 입력 데이터
    self.inputs = tf.placeholder(tf.float32, [None, 28, 28], name="inputs")
    
    # 정답 데이터
    self.targets = tf.placeholder(tf.int32, [None, self.num_classes], name="targets")

    # 노드를 보전할 확률
    self.keep_prob = tf.placeholder(tf.float32, [], "keep_prob")


  def _cnn_init(self):
    with tf.name_scope("cnn_layer"):
         
      convolution_w = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01), name="convolution_w")

      # (?, 28, 28) -> (?, 28, 28, 1)
      inputs = tf.expand_dims(input=self.inputs, axis=-1)
      
      # (?, 28, 28, 1) -> (?, 28, 28, 32)
      conv_layer = tf.nn.conv2d(inputs, convolution_w, strides=[1, 1, 1, 1], padding='SAME', name="conv_layer")
      # (?, 28, 28, 32) -> (?, 28, 28, 32)
      conv_layer = tf.nn.relu(conv_layer)
      
      # (?, 28, 28, 32) -> (?, 14, 14, 32)
      conv_layer = tf.nn.max_pool(conv_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
      # (?, 14, 14, 32) -> (?, 14, 14, 32)
      conv_layer = tf.nn.dropout(conv_layer, keep_prob=self.keep_prob)

      # (?, 14, 14, 32) -> (?, 6272)
      conv_layer = tf.reshape(tensor=conv_layer, shape=[-1, 14*14*32])
      
      # (?, 6272) -> (?, 10)
      fully_connected_layer = tf.layers.dense(inputs=conv_layer, units=10)
    
      self.outputs = fully_connected_layer

  def _predict_init(self):
    # 각 클래스의 분포 값들을 softmax 함수를 이용하여 0~1사이의 값으로 변경
    self.predict_op = tf.nn.softmax(logits=self.outputs, axis=-1)

  def _train_init(self):
    if self.mode == "train":
      with tf.variable_scope("train_layer"):
        # 모델의 출력인 self.outputs 와 self.targets를 비교하여 loss 계산
        self.loss = tf.losses.softmax_cross_entropy(onehot_labels=self.targets, logits=self.outputs)
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.grads = self.optimizer.compute_gradients(self.loss)
        self.train_op = self.optimizer.apply_gradients(self.grads)

<h2>read_file, get_batch 함수</h2>

<pre>
<b>1. read_file(file_path)</b>
  "mnist_train.csv", "mnist_test.csv" 파일을 읽기 위한 함수
  
  read_file(file_path)
  args
    file_path : 읽고자 하는 데이터의 경로
  return
    입력 이미지에 대한 픽셀값과 해당 이미지의 라벨을 담고 있는 리스트
    
  데이터 예시)
    1, 0, 0, 0, 0, ... 0.6, 0.8, 0, 0, ... 0.7, 0.1, 0, 0, ... 0, 0, 0
    라벨, 픽셀값, 픽셀값, ... 픽셀값
</pre>
![대체 텍스트](http://nlp.kangwon.ac.kr/~nlpdemo/mnist_1.png)
<pre>
<b>2. get_batch(datas, batch_size)</b>
  전체 데이터를 batch 단위로 나누어 주기 위한 함수
  
  get_batch(datas, batch_size)
  args
    datas : 입력 이미지에 대한 픽셀값과 해당 이미지의 라벨로 이루어진 리스트
    batch_size : 한번에 학습할 데이터의 개수
  return
    batch 단위로 나뉘어진 데이터 리스트
    
  예시) 
    batch_size = 3 인 경우
  
    total_datas = [ (입력1, 라벨1), (입력2, 라벨2), (입력3, 라벨3), ... ,(입력10, 라벨10) ]
    batches = [
    [ (입력1, 라벨1), (입력2, 라벨2), (입력3, 라벨3)],
    [ (입력4, 라벨4), (입력5, 라벨5), (입력6, 라벨6)],
    [ (입력7, 라벨7), (입력8, 라벨8), (입력9, 라벨9)],
    ]
</pre>

In [0]:
import csv
import numpy as np

# 데이터를 읽고 픽셀값과 라벨을 분리하여 datas 리스트에 저장
def read_file(file_path):
    inFile = open(file_path, "r", encoding="utf8")
    input_file = csv.reader(inFile)

    datas = []
    for line in input_file:
        data, label = np.array(line[1:]), np.array(line[0])
        index = int(label)

        # 픽셀값을 (28, 28) 행렬로 변환
        # 라벨값은 one-hot encoding으로 변환
        # 예시) 2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
        data, label = np.reshape(data, (28, 28)), np.zeros(shape=(10))
        label[index] = 1

        datas.append((data, label))

    inFile.close()

    return datas

# 데이터를 batch 단위로 분할하여 저장
def get_batch(datas, batch_size):
  
  # batches : batch 단위로 저장할 리스트
  # x_datas : 각 batch 단위 별 입력 데이터
  # y_datas : 각 batch 단위 별 정답 데이터
  batches, x_datas, y_datas = [], [], []
  
  for data, label in datas:
    x_datas.append(data)
    y_datas.append(label)

    # x_datas에 담긴 데이터의 개수가 batch_size와 같아지면 
    # x_datas와 y_datas를 batches 리스트에 저장하고 x_datas, y_datas는 비움
    if(len(x_datas) == batch_size):
      batches.append((x_datas, y_datas))
      x_datas, y_datas = [], []

  return batches

<h2>CNN 모델 학습</h2>

<pre>
<b>1. read_file(file_path) 함수를 사용하여 학습 데이터 읽기</b>

<b>2. CNN 모델 객체 선언</b>

<b>3. epoch를 돌때마다 학습 데이터 셔플</b>

<b>3. batch 단위로 학습을 수행</b>
</pre>

In [0]:
import os
import numpy as np
from tqdm import tqdm

def train(flags):
  # 학습 데이터 읽기
  train_datas = read_file(flags["train_data_path"])

  # 모델 객체 선언
  model = CNN(flags)
  
  # tensorflow session 옵션 설정
  # allow_soft_placement=True : 어떤 device를 사용하여 연산할지 명시하지 않은 경우 자동으로 존재하는 디바이스 중에서 하나를 선택
  # gpu_options=tf.GPUOptions(allow_growth=True) : 연산 실행 과정에서 필요한만큼의 gpu 메모리만 사용
  sess_config = tf.ConfigProto(allow_soft_placement=True, 
                               gpu_options=tf.GPUOptions(allow_growth=True))

  # tensorflow를 실행하기 위한 session 
  with tf.Session(config=sess_config) as sess:
    # 그래프 초기화
    sess.run(tf.global_variables_initializer())
    # 학습 파일을 저장거나 불러오기 위한 saver 객체
    saver = tf.train.Saver(max_to_keep=10)

    for epoch in tqdm(range(flags["epoch"])):
      # 학습 데이터 셔플
      np.random.shuffle(train_datas)
      # 학습 데이터를 batch 단위로 분할하여 저장
      batches = get_batch(train_datas, flags["batch_size"])

      losses = []
      # batch 단위로 학습을 진행하며 각 batch 별 loss를 구한다
      # batch 별 loss들의 평균을 구하여 이를 전체 데이터에 대한 loss로 사용
      for train_x, train_y in batches:
          loss, train_op = sess.run([model.loss, model.train_op],
                                    feed_dict={ model.inputs:train_x, 
                                               model.targets:train_y, 
                                               model.keep_prob:flags["keep_prob"] }
                                    )
          losses.append(loss)

      # 학습한 모델 파일 저장
      filename = os.path.join(flags["save_dir"], "model_{}.ckpt".format(epoch+1))
      saver.save(sess, filename)

      print("\tEpoch : {}, Average_Loss : {}".format(epoch+1, np.mean(losses)))

<h2>CNN 모델 평가</h2>

<pre>
<b>1. read_file(file_path) 함수를 사용하여 평가 데이터 읽기</b>

<b>2. CNN 모델 객체 선언</b>

<b>3. tf.train.Saver() 객체를 사용하여 학습한 모델 파일 중에서 가장 많이 학습된 파일로부터 가중치를 불러옴</b>
</pre>

In [0]:
import numpy as np
from tqdm import tqdm

def test(flags):
  # 평가 데이터 읽기
  test_datas = read_file(flags["test_data_path"])

  # 모델 객체 선언
  model = CNN(flags)
  # tensorflow session 옵션 설정
  sess_config = tf.ConfigProto(allow_soft_placement=True, 
                               gpu_options=tf.GPUOptions(allow_growth=True))

  # tensorflow를 실행하기 위한 session 
  with tf.Session(config=sess_config) as sess:
    # 그래프 초기화
    sess.run(tf.global_variables_initializer())
    # 학습 파일을 저장거나 불러오기 위한 saver 객체
    saver = tf.train.Saver()
    
    # 학습한 모델 파일 중에서 가장 많이 학습된 파일로부터 가중치를 불러옴
    print("Read from : " + str(tf.train.latest_checkpoint(flags["save_dir"])))
    saver.restore(sess, tf.train.latest_checkpoint(flags["save_dir"]))

    # 평가 데이터를 batch 단위로 분할하여 저장
    batches = get_batch(test_datas, flags["batch_size"])

    # 정답을 맞춘 개수
    correct_count = 0
    for test_x, test_y in tqdm(batches):
      predict_op = sess.run(model.predict_op,
                            feed_dict={ model.inputs:test_x, model.keep_prob:flags["keep_prob"] }
                            )

      # 모델의 outputs에는 각 클래스에 대한 분포가 저장되어 있고
      # np.argmax 함수를 통하여 가장 확률이 높은 클래스를 선택
      # 예시) 
      #  predict_op = [0,1, 0.3, 0.2] (각각 0일 확률, 1일 확률, 2일 확률)
      #  np.argmax(predict_op) = 1
      outputs, correct = np.argmax(predict_op, axis=-1), np.argmax(test_y, axis=-1)
      correct_count += np.sum(np.equal(outputs, correct))

    print("Accuracy : " + str(100.0*correct_count/(len(batches))))

<h2>모델의 hyper parameter 설정, 학습 및 평가 실행</h2>

<pre>
root_dir : 코드와 데이터가 있는 디렉토리 경로
save_dir : 학습한 모델 파일을 저장할 디렉토리 경로(디렉토리가 존재하지 않을 경우 자동으로 생성)

<b>flags : hyper parameter를 저장할 딕셔너리</b>
  flags.mode = 학습 또는 평가 설정("train" or "test")
  flags.save_dir = 학습한 모델 파일을 저장할 디렉토리 경로
  flags.batch_size = 한번에 학습할 데이터의 개수
  flags.epoch = 학습 횟수
  flags.learning_rate = 학습률
  flags.keep_prob = 노드를 보전할 확률
  flags.num_classes = 분류할 클래스의 개수(0~9, 따라서 10개)
  flags.train_data_path = 학습데이터 파일 경로
  flags.test_data_path = 평가데이터 파일 경로

<b>mode 별 hyper parameter 변경</b>
  학습하는 경우 : mode를 "train"으로 설정, 나머지는 기본 설정 그대로 유지
  평가하는 경우 : mode를 "test"로, batch_size는 1로, keep_prob은 1.0으로 변경
</pre>

In [7]:
import os

if __name__ == "__main__":
  root_dir = "/gdrive/My Drive/colab/cnn"
  save_dir = os.path.join(root_dir, "model")
  if not os.path.exists(save_dir):
      os.makedirs(save_dir)

  flags = {"mode":"train",
           "save_dir":save_dir,
           "batch_size":64,
           "epoch":10,
           "learning_rate":0.001,
           "keep_prob":0.7,
           "num_classes":10,
           "train_data_path":os.path.join(root_dir, "mnist_train.csv"),
           "test_data_path":os.path.join(root_dir, "mnist_test.csv")          
          }
  
  tf.reset_default_graph()
  if(flags["mode"] == "train"):
      train(flags)
  elif(flags["mode"] == "test"):
      flags["batch_size"] = 1  
      flags["keep_prob"] = 1.0
      test(flags)
  else:
      print("Unknown mode")
      exit(0)

  0%|          | 0/10000 [00:00<?, ?it/s]

Read from : /gdrive/My Drive/colab/cnn/model/model_10.ckpt


100%|██████████| 10000/10000 [00:17<00:00, 587.86it/s]

Accuracy : 97.64
